In [1]:
import torch

In [2]:
import numpy as np

In [43]:
a = torch.Tensor(3)

In [44]:
a


 3.2332e+30
 4.5647e-41
-2.4331e-18
[torch.FloatTensor of size 3]

In [45]:
def sigma(x):
    return torch.tanh(x)

In [46]:
def sigma1(x):
    #Does it inplace in the tensor you pass in as input
    return torch.Tensor.tanh_(x)

In [47]:
sigma(a)


 1.0000e+00
 4.5647e-41
-2.4331e-18
[torch.FloatTensor of size 3]

In [48]:
def dsigma(x):
    sig = sigma(x)
    return (1-sig)*(1+sig)

In [49]:
a


 3.2332e+30
 4.5647e-41
-2.4331e-18
[torch.FloatTensor of size 3]

In [50]:
dsigma(a)


 0
 1
 1
[torch.FloatTensor of size 3]

In [56]:
t1 = torch.randn(4)
t2 = torch.randn(4)

In [57]:
t1, t2

(
 -1.5192
 -0.0759
  0.3013
 -0.6904
 [torch.FloatTensor of size 4], 
 -1.0173
  0.0801
  1.5486
  1.1649
 [torch.FloatTensor of size 4])

In [64]:
t1-t2, torch.mul(t1-t2,2)

(
 -0.5019
 -0.1560
 -1.2473
 -1.8552
 [torch.FloatTensor of size 4], 
 -1.0039
 -0.3119
 -2.4946
 -3.7105
 [torch.FloatTensor of size 4])

In [60]:
torch.pow(t1-t2,2)


 0.2519
 0.0243
 1.5558
 3.4419
[torch.FloatTensor of size 4]

In [61]:
torch.sum(torch.pow(t1-t2,2))

5.273967497050762

In [62]:
def loss(v, t):
    return torch.sum(torch.pow(v-t, 2))

In [65]:
def dloss(v, t):
    return torch.mul(v-t, 2)

In [66]:
tmp_x = torch.randn(5)

In [67]:
tmp_x



 0.2057
-0.8277
 0.0074
 1.0358
 0.0797
[torch.FloatTensor of size 5]

In [68]:
def affine(w, b, x):
    return (torch.mv(w, x) + b)

In [78]:
def forward_pass(w1, b1, w2, b2, x):
    s1 = affine(w1, b1, x)
    x1 = sigma(s1)
    s2 = affine(w2, b2, x1)
    x2 = sigma(s2)
    return (x, s1, x1, s2, x2)

In [80]:
torch.mul?

In [83]:
tmp_x= torch.randn(4)
tmp_y= torch.randn(4)

In [84]:
tmp_x, tmp_y

(
 -0.4664
  1.0176
 -0.1147
  2.0917
 [torch.FloatTensor of size 4], 
 -0.7851
  0.4385
  0.1869
  1.1764
 [torch.FloatTensor of size 4])

In [85]:
tmp_x*tmp_y


 0.3662
 0.4462
-0.0214
 2.4605
[torch.FloatTensor of size 4]

In [88]:
tmp_x


-0.4664
 1.0176
-0.1147
 2.0917
[torch.FloatTensor of size 4]

In [89]:
torch.t(tmp_w)


 1  0  0  0  0
 0  1  0  0  0
 0  0  1  0  0
 0  0  0  1  0
 0  0  0  0  1
[torch.FloatTensor of size 5x5]

In [87]:
torch.t(tmp_x.view(-1, 1))


-0.4664  1.0176 -0.1147  2.0917
[torch.FloatTensor of size 1x4]

In [101]:
tmp_x, tmp_x*3

(
 -0.4664
  1.0176
 -0.1147
  2.0917
 [torch.FloatTensor of size 4], 
 -1.3992
  3.0527
 -0.3440
  6.2750
 [torch.FloatTensor of size 4])

In [90]:
tmp_x.view(1, -1)


-0.4664  1.0176 -0.1147  2.0917
[torch.FloatTensor of size 1x4]

In [92]:
def backward_pass(w1, b1, w2, b2, t, x, s1, x1, s2, x2, dl_dw1, dl_db1, dl_dw2, dl_db2):
    dl_dx2 = dloss(t, x2)
    dl_ds2 = dl_dx2*dsigma(s2)
    dl_db2 = dl_ds2
    dl_dx1 = torch.mv(torch.t(w2), dl_ds2)
    dl_dw2 = torch.mm(dl_ds2.view(-1, 1), x1.view(1, -1))
    dl_ds1 = dl_dx1*dsigma(s1)
    dl_db1 = dl_ds1
    dl_dw1 = torch.mm(dl_ds1.view(-1, 1), x.view(1, -1))

In [93]:
import dlc_practical_prologue as prologue

In [114]:
train_input, train_target, test_input, test_target = prologue.load_data(one_hot_labels=True)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [115]:
train_input.shape, train_target.shape

(torch.Size([1000, 784]), torch.Size([1000, 10]))

In [100]:
MULT = 0.9

train_input is torch.cuda.FloatTensor of size 1000x784 (GPU 0)

train_target is torch.cuda.FloatTensor of size 1000x10 (GPU 0) 
It is essentially a one-hot vector but with active unit as 1 and inactive as -1

In [118]:
train_target_modified = train_target*MULT
test_target_modified = test_target*MULT

In [119]:
train_target_modified


-0.9000 -0.9000 -0.9000  ...  -0.9000 -0.9000 -0.9000
 0.9000 -0.9000 -0.9000  ...  -0.9000 -0.9000 -0.9000
-0.9000 -0.9000 -0.9000  ...  -0.9000 -0.9000 -0.9000
          ...             ⋱             ...          
 0.9000 -0.9000 -0.9000  ...  -0.9000 -0.9000 -0.9000
-0.9000 -0.9000 -0.9000  ...  -0.9000 -0.9000 -0.9000
-0.9000 -0.9000 -0.9000  ...  -0.9000 -0.9000 -0.9000
[torch.cuda.FloatTensor of size 1000x10 (GPU 0)]

In [120]:
def custom_randn(sz, mu=0, stdev=1):
    return (stdev*torch.randn(sz) + mu)

In [121]:
HIDDEN_LAYER_SIZE = 50

In [122]:
INPUT_LAYER_SIZE = train_input.shape[1]

In [123]:
INPUT_LAYER_SIZE

784

In [124]:
OUTPUT_LAYER_SIZE = train_target.shape[1]

In [125]:
OUTPUT_LAYER_SIZE

10

In [127]:
STDEV = 1e-3

In [128]:
w1 = custom_randn((INPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE), stdev=STDEV)

In [130]:
w1.shape

torch.Size([784, 50])

In [131]:
w2 = custom_randn((HIDDEN_LAYER_SIZE, OUTPUT_LAYER_SIZE), stdev=STDEV)

In [132]:
b1 = custom_randn((HIDDEN_LAYER_SIZE), stdev=STDEV)

In [133]:
b2 = custom_randn((OUTPUT_LAYER_SIZE), stdev=STDEV)

In [134]:
w2.shape, b1.shape, b2.shape

(torch.Size([50, 10]), torch.Size([50]), torch.Size([10]))

In [135]:
dl_db2 = torch.Tensor(b2.shape).zero_()

In [137]:
dl_db2.shape

torch.Size([10])

In [138]:
dl_db1 = torch.Tensor(b1.shape).zero_()

In [139]:
dl_db1.shape

torch.Size([50])

In [140]:
dl_dw2 = torch.Tensor(w2.shape).zero_()
dl_dw1 = torch.Tensor(w1.shape).zero_()

In [141]:
dl_dw2.shape, dl_dw1.shape

(torch.Size([50, 10]), torch.Size([784, 50]))

In [143]:
ETA = 0.1

In [144]:
NUM_STEPS = train_input.shape[0]

In [147]:
x, s1, x1, s2, x2 = forward_pass(x=train_input[0, ], w1=w1, b1=b1, w2=w2, b2=b2)

RuntimeError: size mismatch at /opt/conda/conda-bld/pytorch_1512386481460/work/torch/lib/THC/generic/THCTensorMathBlas.cu:52

In [ ]:
for k in range(NUM_STEPS):
    x, s1, x1, s2, x2 = forward_pass(x=train_input[0, ], w1=w1, b1=b1, w2=w2, b2=b2)
    w = 